In [1]:
import wikipedia
import re
from pymystem3 import Mystem
from tqdm import tqdm
from multiprocessing import Pool

wikipedia.set_lang('ru')

In [2]:
page_names = ['Трезубец_Посейдона', 'Флаг Барбадоса', 'Биологическое_оружие',
'Блэкберн,_Люк_Прайор', 'Рим', 'Аппиева_дорога_при_закате_солнца']

wikipedia_pages = [wikipedia.page(page_name) for page_name in page_names]

In [7]:
docs = []
m = Mystem()
for page in wikipedia_pages:
    s = re.sub(r'\=\=.*\=\=', ' ', page.content) # remove headers
    s = re.sub(r'[^\w \.]', '', s.lower()) # remove non-letter symbols
    s = [doc.strip() for doc in s.split('.')]
    with Pool(4) as p:
        lemmitized = list(tqdm(p.imap(m.lemmatize, s), total=len(s)))
    # lemmitized = [m.lemmatize(doc) for doc in s]
    docs += [[lemma for lemma in doc if lemma.isalpha() or lemma.isdigit()] for doc in lemmitized]

 17%|█▋        | 30/179 [00:36<02:44,  1.10s/it]

KeyboardInterrupt: 

In [5]:
docs = []
m = Mystem()
def page_to_docs_pipeline(page_content):
    s = re.sub(r'\=\=.*\=\=', ' ', page_content) # remove headers
    s = re.sub(r'[^\w \.]', '', s.lower()) # remove non-letter symbols
    s = [doc.strip() for doc in s.split('.')]
    with Pool(4) as p:
        lemmitized = list(tqdm(p.imap(m.lemmatize, s), total=len(s)))
    return [[lemma for lemma in doc if lemma.isalpha() or lemma.isdigit()] for doc in lemmitized]

from multiprocessing import Pool

with Pool(4) as p:
      docs = list(tqdm(p.imap(page_to_docs_pipeline, wikipedia_pages), total=len(wikipedia_pages)))

# for page in tqdm(wikipedia_pages):
#     s = re.sub(r'\=\=.*\=\=', ' ', page.content) # remove headers
#     s = re.sub(r'[^\w \.]', '', s.lower()) # remove non-letter symbols
#     s = [doc.strip() for doc in s.split('.')]
#     lemmitized = [m.lemmatize(doc) for doc in s]
#     docs += [[lemma for lemma in doc if lemma.isalpha() or lemma.isdigit()] for doc in lemmitized]


0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
queries = [
    'Пейзаж с выжженной равниной увековечил образ вечного города.',
    'Будущего губернатора Кентукки обвиняли в подготовке применения биологического оружия.',
    'Трезубец на жёлто-синем фоне — это флаг Барбадоса.'
]

queries_processed = []
for query in tqdm(queries):
    s = m.lemmatize(query.lower())
    s = [lemma for lemma in doc if lemma.isalpha or lemma.isdigit()]
    queries_processed += s